In [ ]:
import tensorflow as tf
import pandas as pd

Load training data 

In [ ]:
training_data_df = pd.read_csv("./Datasets/sales_data_training.csv", dtype=float)

In [ ]:
training_data_df.head()

Prepare X and y

In [ ]:
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

Load test data 

In [ ]:
test_data_df = pd.read_csv("./Datasets/sales_data_test.csv", dtype=float)

In [ ]:
test_data_df.head()

In [ ]:
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

### Data Preprocessing 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

scale training data 

In [ ]:
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

scale test data 

In [ ]:
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

### Neural Netwok configuration settings 

Model Parameters 

In [ ]:
learning_rate = 0.001
training_epochs = 100

Define how many inputs and outputs

In [ ]:
number_of_inputs = 9
number_of_outputs = 1

Define how many neurons we want in each layer

In [ ]:
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

### Building Computation Graph

Input layer 

In [ ]:
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

Layer 1

In [ ]:
with tf.variable_scope('layer_1'):
    
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)


Layer 2

In [ ]:
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

Layer 3

In [ ]:

with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)


Output Layer

In [ ]:
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

Cost Function 

In [ ]:
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

Optimizer function

In [ ]:
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Training Phase

### Load Model 

In [ ]:
saver = tf.train.Saver()

In [ ]:
session=tf.Session()

In [ ]:
# Instead, load them from disk:
saver.restore(session, "logs/trained_model.ckpt")

In [ ]:
Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})

# Unscale the data back to it's original units (dollars)
Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

real_earnings = test_data_df['total_earnings'].values[0]
predicted_earnings = Y_predicted[0][0]

print("The actual earnings of Game #1 were ${}".format(real_earnings))
print("Our neural network predicted earnings of ${}".format(predicted_earnings))